In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama3"

In [6]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("tell me a joke") 

'Why couldn\'t the bicycle stand up by itself?\n\nBecause it was two-tired!\n\n(Sorry, I know it\'s a bit of a "dad" joke, but I hope it brought a smile to your face!)'

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile!"

In [8]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [9]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago!'

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("doc.pdf")
pages = loader.load_and_split()
pages

invalid pdf header: b'\x923\x00\x00%'


incorrect startxref pointer(1)


[Document(page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 3 \nPETITIONER:\nNARAIN DAS\n        Vs.\nRESPONDENT:\nTHE STATE OF UTTAR PRADESH\nDATE OF JUDGMENT:\n14/09/1960\nBENCH:\nACT:\nAppeal-Forum--Single  Judge of High Court  exercising  civil\njurisdiction  refusing to file complaint-Appeal, if lies  to\nSupreme Court-Code of Criminal Procedure, 1898 (V of  1898),\nss.  195 and 476-B.\nHEADNOTE:\nDuring  the  pendency  of  a  civil  writ  petition  in  the\nAllahabad  High Court, one N moved an application  under  s.\n476,  Code  of criminal Procedure, for  making  a  complaint\nunder  s. 93, Indian Penal Code, against T. A  single  judge\nwho  was  seized  of  the  case  rejected  the  application.\nThereupon  N  presented  an  appeal  against  the  order  of\nrejection of his application before the Supreme Court  under\nS. 476-B, Code of Criminal Procedure.\nHeld,  that the appeal did not lie to the Supreme Court  but\nthat  it lay to the Appellate Bench of the

In [13]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [14]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(page_content="http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 2 of 3 \norder of the learned Judge of the High Court that Narain Das\nhas  filed  this memorandum of appeal under s. 476B  of  the\nCode.   The Registry has submitted the memorandum of  appeal\nwith  a  report  for determining the  question  whether  the\nappeal is competent in this Court.\nSection 476 of the Code is to be found in Ch.  XXXV which is\nheaded  'Proceedings in case of certain  Offences  Affecting\nthe  Administration of Justice'.  Section 476  empowers  any\nCivil, Revenue or Criminal Court, when it is of the  opinion\nthat  it  is expedient in the interests of justice  that  an\ninquiry  should be made into any offence referred to  in  s.\n195(1) (b) or (c) which appears to have been committed in or\nin relation to a proceeding before it, to file a  complaint,\nafter  such  inquiry  as  it  thinks  necessary,  before   a\nMagistrate  of   Class  having jurisdiction.   It  is  clear\ntherefore that 

In [15]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [16]:
questions = [
    "why was the appeal rejected",
    # "How many hours of live sessions?",
    # "How many coding assignments are there in the program?",
    # "Is there a program certificate upon completion?",
    # "What programming language will be used in the program?",
    # "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: why was the appeal rejected
Answer: Based on the context, the appeal was rejected because it was held that an appeal under s. 476B does not lie to this Court (Supreme Court of India) from the decision of a single Judge of a High Court refusing to file a complaint under s. 476 of the Code.

The Court distinguished the case M. S. Sheriff v. The State of Madras, which dealt with an appeal lying to this Court under s. 476B of the Code from an order of a Division Bench of a High Court. Since no appeal lies to the High Court against the decision of a single Judge, and therefore an appeal under s. 476B from an order of the single Judge must lie to the High Court, the present appeal was rejected as it does not lie to this Court.

In other words, the Supreme Court held that the forum of appeal is the High Court of Judicature at Allahabad, and not this Court.

